<a href="https://colab.research.google.com/github/anjalipaladi/Named-Entity-Recognition/blob/main/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
dataset=pd.read_csv('hate_speech.csv')
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.label.value_counts()

In [ ]:
for index,tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"-",tweet)

In [ ]:
import re
def clean_text(text):
  text=re.sub(r'[^a-zA-Z\']',' ',text)
  text=re.sub(r'[^\x00-\x7F]+'," ",text)
  text=text.lower()
  return text

In [ ]:
dataset["clean_text"]=dataset.tweet.apply(lambda x: clean_text(x))

In [ ]:
dataset.head(10)

3.Feature Engineering


In [ ]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
stop=stopwords.words('english')

In [ ]:
def gen_freq(text):
  word_list=[]
  for tw_words in text.split():
    word_list.extend(tw_words)
  word_freq=pd.Series(word_list).value_counts()
  word_freq-=word_freq.drop(stop,errors='ignore')
  return word_freq

In [ ]:
def any_neg(words):
  for word in words:
    if word in ['n','no','non','not'] or re.search(r"\wn't",word):
      return 1
    else:
      return 0

In [ ]:
def any_rare(words,rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

In [ ]:
def is_question(words):
  for word in words:
    if word in ['when','what','how','why','who']:
      return 1
    else:
      return 0

In [ ]:
word_freq=gen_freq(dataset.clean_text.str)
rare_100=word_freq[-100:]
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x: len(x))
dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x: any_rare(x,rare_100))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x: is_question(x))
dataset['char_count']=dataset.clean_text.apply(lambda x: len(x))

In [ ]:
dataset.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
X=dataset[['word_count','any_neg','any_rare','is_question']]
y=dataset.label
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


4.Train an ML model for Text Classifictaion

In [ ]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model=model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [ ]:
model.predict(X_test[5:10])

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test,y_pred)*100,"%")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf=RandomForestClassifier()
clf_rf.fit(X_train,y_train)
rf_pred=clf_rf.predict(X_test).astype(int)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(y_test,rf_pred))
print(confusion_matrix(y_test,rf_pred))
print("Accuracy:",accuracy_score(y_test,rf_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(class_weight='balanced')

logreg.fit(X_train,y_train)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))